This notebook has all functions.
Methods :

Method 1 : Threshold-->Filter-->Erosion-->Dilation
Method 2 : Filter-->Threshold-->Erosion-->Dilation

In [ ]:

BASE_DIR="/Users/Trupti/01-LIDo/02-VijiProject/ImageAnalysis/" 
random.seed(3)

In [16]:
def save_img(folder_path,img_name,labelled_name,folder_name,img_folder):
    directory=folder_path.split('/')[-1].split('.')[0] # to create a folder per experiment to save images
    path= BASE_DIR + "AnalysisMethods/AnalysisResults/AMovie/"+ folder_name + "/" + img_folder + "/" + directory 
    try:
        os.makedirs(path)
    except FileExistsError:
    # directory already exists
        pass
    plt.imsave((path + '/' + labelled_name + '.png'),img_name,dpi=300,cmap='gray')
    

In [ ]:
def save_img_labels(folder_path,intensity_im,labelled_mask,labelled_name,folder_name,num_labels,img_folder):
    directory=folder_path.split('/')[-1].split('.')[0] # to create a folder per experiment to save images
    path= BASE_DIR + "AnalysisMethods/AnalysisResults/AMovie/" + "/"+ folder_name + "/" + img_folder + "/" + directory 
    try:
        os.makedirs(path)
    except FileExistsError:
    # directory already exists
        pass
    for i in range(num_labels):
        indexmean=ndi.mean(intensity_im,labelled_mask,index=i+1)
        labelled_mask[np.where(labelled_mask==i+1)]=indexmean
    #plt.imsave((path + '/' + labelled_name + '.png'),labelled_mask,dpi=300, cmap='Blues')
    
    ###### uncommnet below to save with the colour map key
    #plt.imshow(labelled_mask,cmap="Blues")
    #plt.colorbar()
    #plt.savefig(path + '/' + '%s.png'%(labelled_name),bbox_inches='tight')
    #plt.clf()

In [ ]:
def cytoplasm_signal(img):
  '''
  This function takes an 8bit image,calculates the pixel value for all 4 corners in a 10x10 window 
  and returns its mean.
  '''
  col,row=img.shape
  topLeft=img[0:10, 0:10].flatten() 
  topRight=img[col-10:col,0:10].flatten()
  bottomLeft=img[0:10,col-10:col].flatten() 
  bottomRight=img[col-10:col,col-10:col].flatten()

  mean_array=np.concatenate([topLeft,topRight,bottomLeft,bottomRight])
  mean=np.mean(mean_array)
  
  return(mean)

In [ ]:
def outliers_astrin(df):
    '''
    This functions removes the outliers in the dataframe for astrin ratio
    '''
    Q1 = df['YFP_astrin@kt/YFP_astrin@cyto'].quantile(0.25)
    Q3 = df['YFP_astrin@kt/YFP_astrin@cyto'].quantile(0.75)
    IQR = Q3 - Q1
    df_out= df[~((df['YFP_astrin@kt/YFP_astrin@cyto'] < (Q1 - 1.5 * IQR)) |(df['YFP_astrin@kt/YFP_astrin@cyto'] > (Q3 + 1.5 * IQR)))]
    return(df_out)

In [ ]:
def cenpb(cenpbimg,cenpbim,propList):
    '''
    This function calculates the intensity of cenpb marker on KT
    Localisation of KT in CENPB - spots
    Image Used : cenpb Image
    
    '''
    # Apply prewitt filter to threshold image
    prewitt_im= filters.prewitt(cenpbimg)
    #Apply threshold
    threshold = filters.threshold_yen(prewitt_im)
    #Generate thresholded image
    threshold_image = prewitt_im > threshold

        #Apply erosion to the filtered image followed by dilation to the eroded image

    erosion_im=morphology.binary_erosion(threshold_image, selem=None, out=None)
    dilation_im=morphology.binary_dilation(erosion_im, selem=None, out=None)
    
    labelled_cenpb,num_labels=ndi.label(dilation_im)


    all_props=measure.regionprops_table(labelled_cenpb, intensity_image=cenpbim, properties=propList)#['label','area','min_intensity','max_intensity','mean_intensity']) # intensity image is 16 bit red astrin image
    df_cenpb= pd.DataFrame(all_props)
    return(df_cenpb,dilation_im,labelled_cenpb,num_labels)

In [ ]:
def astrin(particle_mask,astrin_img,astrin_im,propList):
    '''
    This function calculates the astrin intensity on KT
   
    '''
    # find the co-ordinates of the rest of the image without the particles
    # dilation_im is the binary image with particles 1 and bg 0

    rest_of_img_idx=np.where(particle_mask==0)
    # copy astrin image into another for further operations
    astrin_astrin_img= astrin_img.copy()
    astrin_astrin_img[rest_of_img_idx]=0
    # create a mask for the astrin after removing the background. We are interested only in the 
    # particles
    threshold=filters.threshold_otsu(astrin_astrin_img)
    astrin_mask =astrin_astrin_img> threshold # main cell stays black
    # find the intensity of the particles(ASTRIN) without the cytoplasm
    labelled_astrin_mask,num_labels_astrin=ndi.label(astrin_mask)
    all_props=measure.regionprops_table(labelled_astrin_mask, intensity_image=astrin_im, properties=propList)#['label','area','min_intensity','max_intensity','mean_intensity']) # intensity image is 16 bit red astrin image
    df_astrin= pd.DataFrame(all_props)
    return(df_astrin,astrin_mask,labelled_astrin_mask,num_labels_astrin)

In [1]:
def cenpb_cyto_fun(particle_mask,cenpbimg,cenpbim):
    '''
    cenpb-Cyto : cenpb cytoplasmic intensity
    Intensity of cenpb signal cell without the particles (cenpb image)
    '''
    threshold=filters.threshold_mean(cenpbimg)
    cenpb_cyto =cenpbimg > threshold
    cenpb_cyto = morphology.remove_small_objects(cenpb_cyto)
    cenpb_cyto = morphology.remove_small_holes(cenpb_cyto)
    # black idx we need to remove the particles. hence select the particles idx 
    idx = np.where(particle_mask != 0)
    cenpb_cyto[idx]=0 
    labelled_cenpb_cyto,num_labels_cenpb_cyto=ndi.label(cenpb_cyto)
    all_props=measure.regionprops_table(labelled_cenpb_cyto, intensity_image=cenpbim, properties=['label','area','min_intensity','max_intensity','mean_intensity']) # intensity image is 16 bit red astrin image
    df_cenpb_cyto= pd.DataFrame(all_props)
    
    try:
        cenpb_cyto_value=df_cenpb_cyto[df_cenpb_cyto['area']==df_cenpb_cyto['area'].max()]['mean_intensity'].values[0]
        
        return(cenpb_cyto_value)
    except:
        return(np.nan)
    

    

In [ ]:
def astrin_cyto_fun(particle_mask,astrin_img,astrin_im,fname):
    
    '''
    Astrin Cyto : YFP-Astrin cytoplasmic intensity
    Intensity of YFP-Astrin signal cell without the particles (astrin image)
    
    '''
    # mask for the cell
    threshold=filters.threshold_mean(astrin_img)

    astrin_cyto_mask =astrin_img >threshold
    astrin_cyto_mask = morphology.remove_small_objects(astrin_cyto_mask)
    astrin_cyto_mask = morphology.remove_small_holes(astrin_cyto_mask)

    
    #black idx # remove KTs

    kt_idx = np.where(particle_mask != 0)
    astrin_cyto_mask[kt_idx]=0 
                                                     
    labelled_astrin_cyto,num_labels_astrin_cyto=ndi.label(astrin_cyto_mask)
    all_props=measure.regionprops_table(labelled_astrin_cyto, intensity_image=astrin_im, properties=['label','area','mean_intensity']) # intensity image is 16 bit red astrin image
    df_astrin_cyto= pd.DataFrame(all_props)

    try:
        astrincyto=df_astrin_cyto[df_astrin_cyto['area']==df_astrin_cyto['area'].max()]['mean_intensity'].values[0]
        return(astrincyto)
    except:
        
        return(np.nan)
        
        
    

In [17]:
def method_cenpb(folder_path,folder_name):
    '''
      This function takes the folder path of tif images and performs following steps.
      1. Reads the image from the path
      2. Converts the 16bit image to 8 bit
      3. Prewitt Filter-->Yen Threshold-->Erosion-->dilation
  
      For mean intensity calculation, the background noise needs to be filtered from the intensity image. 

    '''
    propList = ['label', 'area','mean_intensity'] 
    df_cenpb_final = pd.DataFrame(columns=propList)
    df_astrin_final= pd.DataFrame(columns=propList)
    
    # set path for images
    astrin_chpath = os.path.join(folder_path,"YFP-astrin","*.tif") #C1 red astrin
    cenpb_chpath = os.path.join(folder_path,"CENPB","*.tif") #C0 green astrin
    # create red astrin image array
    astrin_image_list=[]
    for file in natsorted(glob.glob(astrin_chpath)):
        astrin_image_list.append(file)
     
    k=0
    for file in natsorted(glob.glob(cenpb_chpath)):
        cenpb_im= io.imread(file)  # This is to measure and label the particles
        #Convert an (ImageJ) TIFF to an 8 bit numpy array
        cenpb_img= (cenpb_im / np.amax(cenpb_im) * 255).astype(np.uint8)
        astrin_im = io.imread(astrin_image_list[k])
        astrin_img= (astrin_im / np.amax(astrin_im) * 255).astype(np.uint8)
        
        # get cenpb signal
        df_cenpb,particle_mask,labelled_cenpb,cenpb_labels=cenpb(cenpb_img,cenpb_im,propList)
        cenpb_fname=re.sub(r'^.+/([^/]+)$', r'\1',file)
        #df_nuf2.insert(0,'nuf2_fname',nuf_fname)
        df_cenpb['label']=str(k) +"_"+ df_cenpb['label'].astype(str)
        df_cenpb['fname']=re.sub(r'^.+/([^/]+)$', r'\1',file)
        
        
       
        # get astrin signal
        df_astrin,astrin_mask,labelled_astrin,astrin_labels=astrin(particle_mask,astrin_img,astrin_im,propList)        
        as_fname=re.sub(r'^.+/([^/]+)$', r'\1',astrin_image_list[k])
        
        df_astrin['fname']=re.sub(r'^.+/([^/]+)$', r'\1',astrin_image_list[k])
        df_astrin['label']=str(k) +"_"+ df_astrin['label'].astype(str)
        
        # cytoplasm intensity of cenpb image
        cenpb_cyto=cenpb_cyto_fun(particle_mask,cenpb_img,cenpb_im)
        #cytoplasm intensity for astrin image
        astrin_cyto=astrin_cyto_fun(particle_mask,astrin_img,astrin_im,as_fname)
        # cenpb bg outside the cell
        cenpb_bg=cytoplasm_signal(cenpb_im)
        # astrin bg outside the cell
        astrin_bg=cytoplasm_signal(astrin_im)
        
        df_astrin['YFP_astrin_cyto']=astrin_cyto
        df_cenpb['CENP_B_ds_red_cyto']=cenpb_cyto
        df_astrin['YFP_astrin_bg']=astrin_bg
        df_cenpb['CENP_B_ds_red_bg']=cenpb_bg
        
        df_cenpb_final=pd.concat([df_cenpb_final,df_cenpb],ignore_index=True)        
        df_astrin_final=pd.concat([df_astrin_final,df_astrin],ignore_index=True)
        
        
        # #SAVE THE IMAGES : uncomment to save the images
        labelled_name=re.sub(r'^.+/([^/]+)$', r'\1',file)
        # save cenpb raw image
        save_img(folder_path,cenpb_img,labelled_name,folder_name,"mask_raw_images")
        # save cenpb signal mask
        save_img(folder_path,particle_mask,labelled_name,folder_name,"mask_images")
         # save astrin raw image
        save_img(folder_path,astrin_img,labelled_name,folder_name,"particle_raw_images")
        # save astrin signal mask
        save_img(folder_path,astrin_mask,labelled_name,folder_name,"particle_images")
          # save labelled cenpb masks
        save_img_labels(folder_path,cenpb_im,labelled_cenpb,labelled_name,folder_name,cenpb_labels,"labelled_cenpb_images")
        
        # save labelled astrin masks
        save_img_labels(folder_path,astrin_im,labelled_astrin,labelled_name,folder_name,astrin_labels,"labelled_astrin_images")
       
       
        
        k+=1
  
    return(df_astrin_final,df_cenpb_final)#,astrin_cyto,cenpb_cyto,astrin_bg,cenpb_bg)
 